In [ ]:
# This code snippet sets up the foundational components for a Retrieval-Augmented Generation (RAG) application.
# This is a very common pattern used to build chatbots that can answer questions about specific, private data 
# (in this case, documents about pets).

from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companion known for its loyalty and friendliness",
        metadata={"source":"mamal-pets-doc"},
    ),
    Document(
        page_content="Cats are Independent pets that enjoy on their own",
        metadata={"source":"mamal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginers requiring relatively simply care",
        metadata={"source":"mamal-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent bird capable of mimiking human speech",
        metadata={"source":"mamal-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that needs plenty of space to hop around",
        metadata={"source":"mamal-pets-doc"},
    ),
]

In [2]:
import os
import os                          # for operating system work
from dotenv import load_dotenv     # to load environment variables 
from langchain_groq import ChatGroq # for the chat module

load_dotenv() # loading all the environment variables
groq_api_key = os.getenv("GROQ_API_KEY") # get the groq api key
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN") # get the hugging face access token

llm = ChatGroq(model="Llama3-8b-8192",groq_api_key=groq_api_key)


In [ ]:
# This imports a class that interfaces with Hugging Face models specifically designed for creating embeddings.
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
## We are using Chroma DB for Vector Store
# This imports the Chroma class from langchain_chroma, which is used to create a vector store from documents and embeddings.
from langchain_chroma import Chroma

# This creates a vector store using the documents and embeddings created earlier.
# The vector store allows for efficient similarity search and retrieval of documents based on their embeddings.
vectorstore = Chroma.from_documents(documents=documents,embedding=embeddings)

In [ ]:
# This performs a similarity search in the vector store for documents related to the query "Dogs".
# It returns documents that are most similar to the query based on their embeddings.
vectorstore.similarity_search("Dogs")


In [ ]:
# This performs an asynchronous similarity search in the vector store for documents related to the query "Dogs".
# It returns documents that are most similar to the query based on their embeddings.
await vectorstore.asimilarity_search("Dogs")

In [ ]:
# This performs a similarity search in the vector store for documents related to the query "Dogs".
# It returns documents that are most similar to the query based on their embeddings.    
vectorstore.similarity_search_with_score("Dogs")


LangChain VectorStore Objects do not subclass Runnable, so it cannot be immediatelly integrated into Langchain Expression Language Chain

Langchain Retrievers are runnable, so they implement a standard set of methods (e.g. synchronous and aynchronous invoke and batch operations) and are designed to be incorporated in LCEL chains.

We can create a simple version of this ourselves, without subclassing Retriever. If we choose what method we wish to use to retrieve documents, we can create runnable easily. Below we will build one round of similarity_search method.


In [ ]:
from typing import List
# This imports the Document class from langchain_core.documents, which is used to represent documents in LangChain.
from langchain_core.documents import Document
# This imports the RunnableLambda class from langchain_core.runnables, which allows for creating custom runnables 
# that can be used in LangChain's expression language.
from langchain_core.runnables import RunnableLambda
# This line wraps the similarity_search method in a RunnableLambda. The retriever variable is now a Runnable object.
#    .bind(k=1): This is a powerful feature of LCEL. The .bind() method "binds" a keyword argument to a runnable. 
#     In this case, you are telling the similarity_search function that k (the number of documents to return)
#     should always be 1. This value is now fixed for this specific retriever object.
retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)
# The .batch() method is designed to process multiple inputs at once, in parallel if possible, and return a list of outputs.
# The retriever.batch(["cat", "dog"]) call will perform two separate retrievals.
# This performs a batch retrieval for the queries "cat" and "dog".
retriever.batch(["cat","dog"])


Vectorstore implement an as_retriever method that will generate a Retriever, specially a VectorStore Retriever.
These retrievers include a specific search_type and search_kwargs attributes that identify what methods of the underlying vectorstore to call, and how to parametrize them. For example we can replicate the above with the following


In [11]:
retriever = vectorstore.as_retriever(
    search_type="similarity",  # Specifies the type of search to perform
    search_kwargs={"k": 1}  # Specifies the number of documents to return for each query
)
retriever.batch(["cat","dog"])


[[Document(id='f3e14f5e-1ef8-40ea-8159-337f0e7b3670', metadata={'source': 'mamal-pets-doc'}, page_content='Cats are Independent pets that enjoy on their own')],
 [Document(id='4c75d691-5097-4f13-be2e-a0196d5ede00', metadata={'source': 'mamal-pets-doc'}, page_content='Dogs are great companion known for its loyalty and friendliness')]]

In [ ]:
# Basic RAG example using LangChain Expression Language (LCEL)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

messages = """
Answer this question, using the provided context only.
{question}

Context:
{context}
"""
# first ( and ): The parentheses for the method call ChatPromptTemplate.from_messages().

# second [ and ]: The brackets for the list of messages.

# third ( and ): The parentheses for the tuple representing a single message (role, content).
# List can take multiple tuples and hence list is used.
prompt=ChatPromptTemplate.from_messages ([("human",messages)])
# In a LangChain Expression Language (LCEL) chain, everything inside the dictionary has to be a Runnable.
# The RunnablePassthrough() is a special runnable that simply passes the input through without any modification.
# Its used to pass the "tell me about Dogs as runnable instead of a string."

rag_chain = {"context":retriever,"question":RunnablePassthrough()} | prompt | llm
response = rag_chain.invoke("tell me about Dogs")
print(response.content)

According to the provided context, Dogs are great companions known for their loyalty and friendliness.
